# Import Stuff

In [1]:
import requests
import pandas as pd
from sqlalchemy import create_engine

In [2]:
API_KEY = "74M88OXCGWTNUIV9" 

___

# Stock Data

In [ ]:
# Alpha Vantage API details
API_KEY = "74M88OXCGWTNUIV9"  # Replace with your API key
symbol = "AAPL"  # Example: Apple Inc.
url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={API_KEY}"

# Fetch data from Alpha Vantage
response = requests.get(url)
data = response.json()

# Parse JSON into DataFrame
time_series = data.get("Time Series (Daily)", {})
df = pd.DataFrame.from_dict(time_series, orient="index")
df.columns = ["open", "high", "low", "close", "volume"]
df.index = pd.to_datetime(df.index)
df = df.sort_index()

# Convert columns to numeric
df = df.apply(pd.to_numeric)

# Store in SQLite database
engine = create_engine("sqlite:///stocks.db")
df.to_sql(f"{symbol}_daily", engine, if_exists="replace")

print(f"Data for {symbol} successfully stored in database!")


____

___

# Currency Pair Data

In [ ]:

import requests
import pandas as pd
from sqlalchemy import create_engine

# Alpha Vantage API details
#API_KEY = "YOUR_API_KEY"  # Replace with your API key
from_symbol = "EUR"  # Base currency
to_symbol = "USD"    # Quote currency

# API URL for daily FX data
url = f"https://www.alphavantage.co/query?function=FX_DAILY&from_symbol={from_symbol}&to_symbol={to_symbol}&apikey={API_KEY}&outputsize=full"

# Fetch data
response = requests.get(url)
data = response.json()

# Parse JSON into DataFrame
time_series = data.get("Time Series FX (Daily)", {})
df = pd.DataFrame.from_dict(time_series, orient="index")
df.columns = ["open", "high", "low", "close"]
df.index = pd.to_datetime(df.index)
df = df.sort_index()

# Convert columns to numeric
df = df.apply(pd.to_numeric)

# ✅ Save to SQLite database
engine = create_engine("sqlite:///forex.db")
table_name = f"{from_symbol}_{to_symbol}_daily"
df.to_sql(table_name, engine, if_exists="replace")

# ✅ Save to CSV file
csv_file = f"{from_symbol}_{to_symbol}_daily.csv"
df.to_csv(csv_file)

print(f"Daily data for {from_symbol}/{to_symbol} stored in database and saved as {csv_file}!")


____

## Multi Currency Pair

In [ ]:

import requests
import pandas as pd
from sqlalchemy import create_engine
import time

# Alpha Vantage API details
#API_KEY = "YOUR_API_KEY"  # Replace with your API key
API_KEY = "74M88OXCGWTNUIV9"
# List of currency pairs (base, quote)
currency_pairs = [
    ("EUR", "USD"),
    ("GBP", "USD"),
    ("USD", "JPY"),
    ("AUD", "USD"),
    ("USD", "CAD")
]

# Create SQLite database connection
engine = create_engine("sqlite:///forex.db")

# Function to fetch and save data
def fetch_and_save_fx(from_symbol, to_symbol, function, interval=None):
    if function == "FX_INTRADAY":
        url = f"https://www.alphavantage.co/query?function={function}&from_symbol={from_symbol}&to_symbol={to_symbol}&interval={interval}&apikey={API_KEY}&outputsize=full"
    else:
        url = f"https://www.alphavantage.co/query?function={function}&from_symbol={from_symbol}&to_symbol={to_symbol}&apikey={API_KEY}&outputsize=full"
    
    response = requests.get(url)
    data = response.json()
    
    # Determine key based on function
    if function == "FX_INTRADAY":
        key = f"Time Series FX ({interval})"
    else:
        key = "Time Series FX (Daily)"
    
    time_series = data.get(key, {})
    if not time_series:
        print(f"⚠ No data returned for {from_symbol}/{to_symbol}. Skipping...")
        return
    
    # Convert to DataFrame
    df = pd.DataFrame.from_dict(time_series, orient="index")
    df.columns = ["open", "high", "low", "close"]
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    df = df.apply(pd.to_numeric)
    
    # Save to DB
    table_name = f"{from_symbol}_{to_symbol}_{'hourly' if function=='FX_INTRADAY' else 'daily'}"
    df.to_sql(table_name, engine, if_exists="replace")
    
    # Save to CSV
    csv_file = f"{table_name}.csv"
    df.to_csv(csv_file)
    
    print(f"✅ {table_name} saved to DB and CSV!")

# Loop through currency pairs
for from_symbol, to_symbol in currency_pairs:
    print(f"Fetching hourly and daily data for {from_symbol}/{to_symbol}...")
    
    # Hourly data
    fetch_and_save_fx(from_symbol, to_symbol, "FX_INTRADAY", interval="60min")
    
    # Daily data
    fetch_and_save_fx(from_symbol, to_symbol, "FX_DAILY")
    
    # Respect API rate limit (5 requests per minute)
    time.sleep(15)

print("✅ All currency pairs processed successfully!")


___

In [1]:

import pandas as pd
from sqlalchemy import create_engine
import plotly.graph_objects as go

# Connect to SQLite database
engine = create_engine("sqlite:///forex.db")

# Load data from a specific table (example: EUR/USD daily)
table_name = "USD_CAD_daily"
query = f"SELECT * FROM '{table_name}'"
df = pd.read_sql(query, engine)

# Convert date column to datetime if needed
df['index'] = pd.to_datetime(df['index'])
df.set_index('index', inplace=True)

# Create candlestick chart
fig = go.Figure(data=[go.Candlestick(
    x=df.index,
    open=df['open'],
    high=df['high'],
    low=df['low'],
    close=df['close']
)])

fig.update_layout(title="EUR/USD Daily Candlestick Chart", xaxis_rangeslider_visible=False)
fig.show()


___

___

# Check The DataBase

In [ ]:
from sqlalchemy import create_engine, inspect

# Connect to your SQLite database
engine = create_engine("sqlite:///forex.db")

# Create an inspector
inspector = inspect(engine)

# List all tables
tables = inspector.get_table_names()

print("Tables in your database:")
for t in tables:
    print(f" - {t}")


___

___

# Connect To The DataBase

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Connect to SQLite database
engine = create_engine("sqlite:///forex.db")

# Table name you saved earlier
table_name = "EUR_USD_Daily"

# Read from the table
df = pd.read_sql(f"SELECT * FROM {table_name}", engine)

# Convert index column (if needed)
if "index" in df.columns:
    df["index"] = pd.to_datetime(df["index"])
    df = df.rename(columns={"index": "timestamp"})
    df = df.set_index("timestamp")

print(df.head())
print(f"\nRows Loaded: {len(df)}")


___